In [4]:
from compChem.Hartree_Fock import Molecule # package is available on github
from scipy.linalg import eigh, sqrtm
import numpy as np
import psi4

In [5]:
def basischanger(c, f):
        """
        allows a change in basis
        
        input:
        c: transformation matrix
        f: operator to be transformed
        """
        return np.einsum("ij,jm,mn->in", c.T, f, c, optimize=True)

In [54]:
class ConstrainedMolecule:
    def __init__(self, geom_file):
        """initiation method, will take in a geometry"""
        #this parameter contains all parameters and methods from the Molecule class
        self.id = Molecule(geom_file)
        self.id.setGuess()
        
        #getting the transformation matrices
        s_eigh = eigh(self.id.overlap)
        s_diag = np.diag(s_eigh[0]**(-1/2))
        self.x = s_eigh[1].dot(s_diag).dot(s_eigh[1].T)
        #needed to transfrom from old to new basis, in this basis, S will equal unity, does not conserve inner product
        self.transform = np.linalg.inv(self.x)
        
        #getting the initial density matrices
        a = self.id.getDensityMatrix("alpha")
        b = self.id.getDensityMatrix("beta")
        a_trans = np.einsum("pq, qr, rs->ps", self.transform, a, self.transform.T, optimize=True)
        b_trans = np.einsum("pq, qr, rs->ps", self.transform, b, self.transform.T, optimize=True)
        self.pMatrix = (a_trans + b_trans)/2
        
    
    def setChargeDensity(self, new_matrix):
        """sets the pMatrix to a new value"""
        self.pMatrix = new_matrix
        
        
    def fock_Alternator(self):
        """
        alters a fock operator to follow CUHF, this entails transformation to a basis where S is unity and P is diagonal, also
        we will change the cv blocks
        
        """
        #get the current matrices
        f_a = self.id.displayFockMatrix("alpha")
        f_b = self.id.displayFockMatrix("beta")
        
       
        
        #we need a closed shell type operator, it is already in the correct basis
        f_cs = (f_a + f_b)/2
        
        
        #amount of paired electrons if the number of alpha electrons is greater then or equal to the number of beta electrons
        cc = self.id.beta
        
        #amount of unpaired electrons
        oo = self.id.alpha - self.id.beta
        
        #alter first blocks
        f_a[:cc + oo, cc + oo:] = f_cs[:cc + oo, cc + oo:]
        f_a[cc + oo:, :cc + oo] = f_cs[cc + oo:, :cc + oo]
        
        f_b[:cc + oo, cc + oo:] = f_cs[:cc + oo, cc + oo:]
        f_b[cc + oo:, :cc + oo] = f_cs[cc + oo:, :cc + oo]
        
        
        return f_a, f_b
        
    def iteratinator(self):
        """adapted version of the original iterator function in the compChem.Hartree_Fock Molecule object"""
        #we will set a guess using a CUHF operator
        f_a = self.id.getDensityMatrix("alpha")
        f_b = self.id.getDensityMatrix("beta")
        
        #transfrom to basis where S is unity
        f_a_trans = np.einsum("pq, qr, rs->ps", self.transform, f_a, self.transform.T, optimize=True)
        f_b_trans = np.einsum("pq, qr, rs->ps", self.transform, f_b, self.transform.T, optimize=True)
        
        #transfrom to basis where P is diagonal, does conserve inner product
        c = eigh(self.pMatrix)[1]
        f_a_no = np.einsum("pq, qr, rs->ps", c.T, f_a_trans, c, optimize=True)
        f_b_no = np.einsum("pq, qr, rs->ps", c.T, f_b_trans, c, optimize=True)
        
        self.id.setGuess(f_a_no, "alpha")
        self.id.setGuess(f_b_no, "beta")
        
        f_a, f_b = self.fock_Alternator()

        self.id.setGuess(f_a, "alpha")
        self.id.setGuess(f_b, "beta")
        
        # setting up entry parameters for the while loop
        E_new = 0  
        E_old = 0
        convergence = False

        # step 2: start iterating
        itercount = 0
        while not convergence and itercount < 1000:

            # calculating block: calculates energies
            E_new = self.id.getElectronicEnergy()
            E_total = self.id.getTotalEnergy()

            # generating block: generates new matrices UHF: account for alpha and beta
            f_a_n, f_b_n = self.fock_Alternator()
            self.id.setGuess(f_a_n, "alpha")
            self.id.setGuess(f_b_n, "beta") 
            self.updatePMatrix()

            # comparing block: will answer the "Are we there yet?" question
            if abs(E_old - E_new) < self.id.converge:
                convergence = True


            # maintenance block: keeps everything going
            print(f"iteration: {itercount}, E_tot: {E_total: .8f}, E_elek: {E_new: .8f}, deltaE: {E_new - E_old: .8f}")
            E_old = E_new

            itercount += 1
        
        return E_total
    
    
    def updatePMatrix(self):
        """updates pMatrix to current self.id densitymatrices"""
        a = self.id.getDensityMatrix("alpha")
        b = self.id.getDensityMatrix("beta")
        self.pMatrix = (a + b)/2

In [55]:
psi4.set_options({"BASIS": "cc-pvdz"})
h1 = ConstrainedMolecule("""
H 0 0 0
H 0 0.86602540378 0.5
H 0 0 1
units angstrom""")
h1.id.setConvergence(1e-10)
h1.iteratinator()

iteration: 0, E_tot: -1.34936023, E_elek: -2.93689187, deltaE: -2.93689187
iteration: 1, E_tot: -1.43991504, E_elek: -3.02744667, deltaE: -0.09055480
iteration: 2, E_tot: -1.44301545, E_elek: -3.03054708, deltaE: -0.00310041
iteration: 3, E_tot: -1.44457797, E_elek: -3.03210960, deltaE: -0.00156252
iteration: 4, E_tot: -1.44419626, E_elek: -3.03172789, deltaE:  0.00038171
iteration: 5, E_tot: -1.44361003, E_elek: -3.03114166, deltaE:  0.00058623
iteration: 6, E_tot: -1.44312098, E_elek: -3.03065261, deltaE:  0.00048905
iteration: 7, E_tot: -1.44278374, E_elek: -3.03031537, deltaE:  0.00033724
iteration: 8, E_tot: -1.44256963, E_elek: -3.03010126, deltaE:  0.00021411
iteration: 9, E_tot: -1.44243985, E_elek: -3.02997149, deltaE:  0.00012977
iteration: 10, E_tot: -1.44236338, E_elek: -3.02989501, deltaE:  0.00007648
iteration: 11, E_tot: -1.44231909, E_elek: -3.02985072, deltaE:  0.00004429
iteration: 12, E_tot: -1.44229368, E_elek: -3.02982532, deltaE:  0.00002540
iteration: 13, E_tot: 

-1.4422571044514227

In [16]:
psi4.set_options({"BASIS": "cc-pvdz"})
h_test = ConstrainedMolecule("""
H 0 0 0
H 0 0.86602540378 0.5
H 0 0 1
units angstrom""")

In [18]:
s = h_test.id.overlap
p = h_test.pMatrix
s_eigh = eigh(s)
s_diag = np.diag(s_eigh[0]**(-1/2))
x = s_eigh[1].dot(s_diag).dot(s_eigh[1].T)
x.T.dot(s).dot(x)

array([[ 1.00000000e+00, -2.75821033e-15, -7.25114413e-16,
        -9.59376278e-18, -4.85722573e-17, -5.34294831e-16,
         2.98372438e-16, -2.32452946e-16,  3.08507328e-18,
         3.67761377e-16, -3.12250226e-16,  3.46944695e-18,
        -2.44596010e-16,  3.65580378e-18, -3.60822483e-16],
       [-3.01191364e-15,  1.00000000e+00, -5.44703171e-16,
         2.59107788e-16,  6.72205347e-18,  1.16053001e-15,
        -1.19088767e-15, -8.47846099e-16, -3.52038566e-17,
        -1.04083409e-17,  1.73645820e-15, -1.19522447e-15,
        -1.13277443e-15,  3.05825019e-17,  6.93889390e-17],
       [-7.56339436e-16, -5.96744876e-16,  1.00000000e+00,
        -9.72211839e-17, -9.02056208e-17, -5.55111512e-17,
        -9.02056208e-17,  3.72098186e-16,  2.55856624e-17,
        -2.22044605e-16, -9.71445147e-17, -9.64506253e-16,
         2.68882139e-16,  4.05842345e-17,  8.32667268e-17],
       [-9.59376278e-18,  2.59107788e-16, -9.72211839e-17,
         1.00000000e+00, -9.79790736e-16,  4.53916406

In [121]:
a = h_test.id.getDensityMatrix('alpha')
b = h_test.id.getDensityMatrix('beta')
x_i = np.linalg.inv(x)
x_i_t = np.linalg.inv(x.T)
a_trans = x_i.dot(a).dot(x_i_t)
b_trans = x_i.dot(b).dot(x_i_t)
p = (a_trans + b_trans)/2
c = eigh(p)
c

(array([-6.00687650e-18, -4.75866979e-18, -3.31473633e-18, -2.27268212e-18,
        -2.20868761e-18, -7.67134458e-33,  4.58363817e-34,  1.80030792e-33,
         5.22874328e-19,  1.93073916e-17,  4.00913763e-17,  6.23349833e-17,
         4.44089210e-16,  5.00000000e-01,  1.00000000e+00]),
 array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          6.16588591e-01,  6.31105095e-01, -4.70664284e-01],
        [ 1.28778561e-01, -1.33799819e-02, -9.08902394e-02,
          2.20571569e-01, -4.32896469e-02,  3.40471047e-15,
         -1.87520514e-16, -1.51128824e-15, -1.38953325e-01,
         -1.03370758e-01,  2.25640737e-01,  6.14706453e-01,
         -5.36766113e-01,  3.19698738e-01, -2.74506401e-01],
        [-1.09393643e-01,  3.48007069e-01, -2.88241726e-02,
          2.71966661e-02,  3.98216954e-01, -1.693

Up untill here everything should be correct (density matrices are idempotent, p matrix has correct eigenvalues). The area below is still under construction.

In [90]:
f_a = h_test.id.displayFockMatrix('alpha')
f_b = h_test.id.displayFockMatrix('beta')

In [125]:
f_a_trans = x.dot(f_a).dot(x.T)
f_a_no = c[1].T.dot(f_a_trans).dot(c[1])
f_b_trans = x.dot(f_b).dot(x.T)
f_b_no = c[1].T.dot(f_b_trans).dot(c[1])

This expression seems a bit too simple. Nevertheless, according to Tsuchimochi & Scuseria it should hold.

In [133]:
f_cs = (f_a_no + f_b_no)/2